In [ ]:
 # This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/hotel-id-to-combat-human-trafficking-2022-fgvc9/test_images/'):
    for filename in filenames:
        print("filename: ", filename)
#         print("middle::", _)
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential



In [ ]:
# os.mkdir('../output/kaggle/working/checkpoint/')
# ! mkdir checkpoint
!pwd

In [ ]:
IMG_SIZE = 224
BATCH_SIZE = 32
TRAIN_DS_PATH = '../input/hotel-id-to-combat-human-trafficking-2022-fgvc9/train_images/'
SEED = 52
SPLIT = 0.2 
CHECKPOINT_FNAME = 'ckpt_effNet.h5'

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(validation_split=SPLIT, 
                           shear_range=0.2,
                           zoom_range=0.2,
                           horizontal_flip=True,
                           rotation_range=0.1,)
val_datagen = ImageDataGenerator(validation_split=SPLIT, 
                           shear_range=0.2,
                           zoom_range=0.2,
                           horizontal_flip=True,
                           rotation_range=0.1,)

train_ds = train_datagen.flow_from_directory(
    TRAIN_DS_PATH, 
    subset='training', 
    seed=SEED,
    target_size=(IMG_SIZE, IMG_SIZE), 
    batch_size=BATCH_SIZE,
#     classes=['100055', '100143', '100206', '1007', '10191']
)
val_ds = val_datagen.flow_from_directory(
    TRAIN_DS_PATH, 
    subset='validation', 
    seed=SEED,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
#     classes=['100055', '100143', '100206', '1007', '10191']
)
# test_ds = tfds.folder_dataset.ImageFolder('../input/hotel-id-to-combat-human-trafficking-2022-fgvc9/test_images/')
# train_mask =  tf.keras.utils.image_dataset_from_directory('../input/hotel-id-to-combat-human-trafficking-2022-fgvc9/train_masks/')

In [ ]:

num_classes = train_ds.num_classes
train_ds.labels
print(num_classes) 

In [ ]:
# AlexNet
# model = keras.models.Sequential([
#     layers.Rescaling(1./255, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
#     keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu'),
#     keras.layers.BatchNormalization(),
#     keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
#     keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
#     keras.layers.BatchNormalization(),
#     keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
#     keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
#     keras.layers.BatchNormalization(),
#     keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
#     keras.layers.BatchNormalization(),
#     keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
#     keras.layers.BatchNormalization(),
#     keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
#     keras.layers.Flatten(),
#     keras.layers.Dense(4096, activation='relu'),
#     keras.layers.Dropout(0.5),
#     keras.layers.Dense(4096, activation='relu'),
#     keras.layers.Dropout(0.5),
#     keras.layers.Dense(num_classes, activation='softmax')
# ])

In [ ]:
# # Transfer learning
# base_model = tf.keras.applications.InceptionResNetV2(
#                      include_top=False,
#                      weights='imagenet',
#                      input_shape=(IMG_SIZE,IMG_SIZE,3)
#                      )
  
# base_model.trainable=False

# model = tf.keras.Sequential([ 
#         base_model,   
#         tf.keras.layers.BatchNormalization(renorm=True),
#         tf.keras.layers.GlobalAveragePooling2D(),
#         tf.keras.layers.Dense(512, activation='relu'),
#         tf.keras.layers.Dense(256, activation='relu'),
#         tf.keras.layers.Dropout(0.5),
#         tf.keras.layers.Dense(128, activation='relu'),
#         tf.keras.layers.Dense(num_classes, activation='softmax')
#     ])

In [ ]:
# New Model
from tensorflow.keras.applications import EfficientNetB0

EffNet = EfficientNetB0(include_top=False,
                             weights='imagenet',
                             input_tensor=None,
                             input_shape=([IMG_SIZE, IMG_SIZE, 3]),
                             pooling='avg'
                       )

model = tf.keras.Sequential()
model.add(EffNet)
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(num_classes, activation='softmax'))

In [ ]:
# opt = keras.optimizers.ADAM(learning_rate=0.001)
checkpoints = keras.callbacks.ModelCheckpoint(filepath=CHECKPOINT_FNAME, verbose=1, save_best_only=False, save_freq=50,save_weights_only=True)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
model.compile(optimizer='Adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
epochs=5
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs, 
  steps_per_epoch = len(train_ds.labels)/BATCH_SIZE,
  validation_steps = len(val_ds.labels)/BATCH_SIZE,
  callbacks = [checkpoints,early_stop]
#   verbose = 2,
#   batch_size = batch_size
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
# code_test=model.predict(test_ds)
# test_ds.class_indices
# train_ds.classes

class_indices = {k: v for v, k in enumerate(train_ds.classes)}
# class_indices

In [ ]:
#create test ds
import os
test_ds_path = []
for dirname, _, filenames in os.walk('../input/hotel-id-to-combat-human-trafficking-2022-fgvc9/test_images/'):
    path_dict = {}
    for filename in filenames:
        
#         print(os.path.join(dirname, filename))
        path_dict[filename] = os.path.join(dirname, filename)
    test_ds_path.append(path_dict)
# print("test_ds_path::", test_ds_path)
# os.system("cp ../input/hotel-id-to-combat-human-trafficking-2022-fgvc9/test_images/abc.jpg ../input/hotel-id-to-combat-human-trafficking-2022-fgvc9/")

In [ ]:
def get_top5(oneD_list):
    return np.argpartition(oneD_list,-5)[-5:]

def listOfTop5(twoD_list):
    return [get_top5(row) for row in twoD_list]

key_list = list(class_indices.keys())
val_list = list(class_indices.values())

def replaceWithHotelID(oneRow):
    return [key_list[i] for i in oneRow]

def replace2DwthHotelID(twoDArray):
#     print('twoDArray::', twoDArray)
    return [replaceWithHotelID(row) for row in twoDArray]

#Single Image Prediction Code
def single_image_prediction(submission_filepath):
    
        submission = tf.keras.preprocessing.image.load_img(
            submission_filepath, 
            grayscale=False, 
            color_mode="rgb", 
            target_size=(IMG_SIZE, IMG_SIZE), 
            interpolation="nearest"
        )
#         print("submission::", submission)

        submission_arr = tf.keras.preprocessing.image.img_to_array(submission)
#         print("submission_arr::", submission_arr)
        submission_arr = np.array([submission_arr])  # Convert single image to a batch.
        predictions = model.predict(submission_arr)
#         print('predictions::', predictions)
        return replace2DwthHotelID(listOfTop5(predictions))

In [ ]:
# single_image_prediction('../input/hotel-id-to-combat-human-trafficking-2022-fgvc9/test_images/abc.jpg')

prediction_list = []

for path_dict in test_ds_path:
    prediction_dict = {}
    for image, path in path_dict.items():
        prediction_dict["image_id"] = image
#         print([int(id) for id in single_image_prediction(path)[0]])
#         print("single_image_prediction(path)::",single_image_prediction(path))
        single_image_prediction(path)[0] = [int(id) for id in single_image_prediction(path)[0]]
#         single_image_prediction(path)[0] = single_image_prediction(path)[0].reverse()
        pred_list = single_image_prediction(path)[0].copy()
        pred_list.reverse()
        
#         print("pred_list(path)[0]:::", pred_list)
#         hotel_ids = ' '.join(single_image_prediction(path)[0])
#         prediction_dict["hotel_id"] = [int(id) for id in single_image_prediction(path)[0]]
#         prediction_dict["hotel_id"] = [str(list(l)).strip("[]").replace(",","") for l in [int(id) for id in single_image_prediction(path)[0]]]
#         prediction_dict["hotel_id"] = ' '.join( prediction_dict["hotel_id"])
#         print("prediction_list::",  *prediction_dict["hotel_id"])
        prediction_dict["hotel_id"] = [str(list(l)).strip("[]").replace(",", "").replace("'", "") for l in [pred_list]]
        prediction_list.append(prediction_dict.copy())


prediction_df = pd.DataFrame(prediction_list)

# print("prediction_list::",  prediction_list)
prediction_df["hotel_id"] = prediction_df["hotel_id"].apply(lambda m: str(m).strip("[]").replace("'", ""))
prediction_df

In [ ]:
prediction_df.to_csv('submission.csv', index=False)
# prediction_df

In [ ]:
# sample_df = pd.read_csv('/kaggle/input/hotel-id-to-combat-human-trafficking-2022-fgvc9/sample_submission.csv')

In [ ]:
# sample_df.info()

In [ ]:
# prediction_df.info()

In [ ]:
np.partition([3, 4, 1, 2,6,8,0], 5)
